In [1]:
import os
from datetime import datetime
# Make logging folder named according to current time
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = './logs/'f"log_{start_time_str}"
os.makedirs(log_dir)

# Initialize logger that adds to text file in current logging folder
from logging_utils import *

init_logger(save_dir=log_dir)
logger = get_logger()




In [2]:
import pathlib
import subprocess
# Get the absolute path to your repository, 
# no matter where you are running this code from

repo_path = os.getcwd() 


git_branch = subprocess.check_output(
    ["git", "-C", repo_path, "rev-parse", "--abbrev-ref", "HEAD"]).strip().decode('UTF-8')

git_commit_short_hash = subprocess.check_output(
    ["git", "-C", repo_path, "describe", "--always"]).strip().decode('UTF-8')


In [3]:
import numpy as np

dim=200
x0=np.squeeze(np.zeros((dim,1)))
x0Type='Zero Vector'

FD_StencilMat=np.zeros((dim,dim))
b = np.ones(dim-1)
c = -2*np.ones(dim)
np.fill_diagonal(FD_StencilMat[1:], b)
np.fill_diagonal(FD_StencilMat[:,1:], b)
np.fill_diagonal(FD_StencilMat, c)
A=FD_StencilMat
AType='1D Laplacian'


retrain_freq=10
e1 = 1e-8
e2 = 1e-10
nmax_iter = 50
restart   = 64

sigma=0.02
DomainL=-5.0
DomainR=5.0



n_steps =500
InputDim=dim
HiddenDim=dim
HiddenDim2=dim
OutputDim=dim

In [4]:
# Initial log message
logger.info("Benchmark 200 dim problem. 2 Layer linear Conv Net. Fully linear network. SGD ")
#  The source is a narrow gaussian that depends on the integer 'time' in a nonlinear way. The gaussian is on the interval [-5,5], and constrained to move on [-3,3]. ")

2020-05-18 12:38:08 [INFO] <ipython-input-4-2172107a22db> 2 - Benchmark 200 dim problem. 2 Layer linear Conv Net. Fully linear network. SGD 


In [5]:
# Set up parameter class
class MyParamsClass():
    def __init__(self, AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash):
        self.AType = AType
        self.dim = dim
        self.nmax_iter = nmax_iter
        self.restart = restart
        self.n_steps =n_steps
        self.retrain_freq=retrain_freq
        self.e1=e1
        self.e2=e2
        self.x0Type=x0Type
        self.InputDim=InputDim
        self.HiddenDim=HiddenDim
        self.HiddenDim2=HiddenDim2
        self.OutputDim=OutputDim
        self.sigma=sigma
        self.DomainL=DomainL
        self.DomainR=DomainR
        self.git_branch=git_branch
        self.git_commit_short_hash=git_commit_short_hash

In [6]:
import os
import json

params = MyParamsClass(AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash)
sorted_params_dict = {k: params.__dict__[k] for k 
                      in sorted(params.__dict__.keys())}

params_filepath = log_dir+'/params.json'
json.dump(sorted_params_dict, open(params_filepath, 'w'), indent=4)

In [7]:
from src_dir import *

In [8]:
# @nn_preconditioner(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
# def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug):
#     return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [9]:
@timer
@nn_preconditioner_timed(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
def MLGMRES(A, b, x0, e, nmax_iter,Err_list,IterErr0_AVG,ProbCount,restart,debug,refine):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)
    

In [10]:
@timer
def GMRES_timed(A, b, x0, e, nmax_iter,restart,debug):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [ ]:
Err_list=[]
IterErr0_AVG=0.0

debug=True

refine1=False
refine2=True

run_time=0.0
run_time_ML=0.0
trainTime_total=0.0
forwardTime_Total=0.0
run_time_ML_NoAdd=0.0


x=np.linspace(DomainL,DomainR,dim)



for ProbIdx in range(n_steps):

    loc=3.0*np.sin(0.5*ProbIdx*np.abs(np.cos(0.5*ProbIdx)))
    b=Gauss_pdf(x,loc,sigma)
    b_norm=np.linalg.norm(b)



    OutList1,run_time1_ML=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,Err_list,IterErr0_AVG,ProbIdx,restart,debug,refine1)  # normalize b for optimal NN performance.
    Out=OutList1[0]
    Err_list=OutList1[1]
    IterErr0_AVG=OutList1[2]
    forwardTime1=OutList1[4]


    OutList2,run_time2_ML=MLGMRES(A, b/b_norm, Out[-10], e2, nmax_iter,Err_list,IterErr0_AVG,ProbIdx,restart,debug,refine2)
    Out2=OutList2[0]
    Err_list=OutList2[1]
    IterErr0_AVG=OutList2[2]
    trainTime=OutList2[3]
    forwardTime2=OutList2[4]

    forwardTime_Total=forwardTime1+forwardTime2+forwardTime_Total
    trainTime_total=trainTime+trainTime_total
    run_time_ML=run_time1_ML+run_time2_ML+run_time_ML




    NonML_Out1,run_time1=GMRES_timed(A, b/b_norm, x0, e1, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    NonML_Out2,run_time2=GMRES_timed(A, b/b_norm, NonML_Out1[-10], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    run_time=run_time1+run_time2+run_time
    
    print(ProbIdx)


logger.info("Runtime of Non-decorated version is:")
logger.info(run_time)

logger.info("Runtime of MLGMRES decorator is:")
logger.info(run_time_ML)


logger.info("Runtime of training (backprop) is:")
logger.info(trainTime_total)

logger.info("Runtime of forward function is:")
logger.info(forwardTime_Total)


0
1
2
loss: 1141065.625 0
loss: 1140758.0 1
loss: 1140450.5 2
loss: 1140140.0 3
loss: 1139825.5 4
loss: 1139508.5 5
loss: 1139186.75 6
loss: 1138860.0 7
loss: 1138526.875 8
loss: 1138186.25 9
loss: 1137836.0 10
loss: 1137475.5 11
loss: 1137101.75 12
loss: 1136713.0 13
loss: 1136309.125 14
loss: 1135888.0 15
loss: 1135447.25 16
loss: 1134984.375 17
loss: 1134496.625 18
loss: 1133982.0 19
loss: 1133437.75 20
loss: 1132860.25 21
loss: 1132246.25 22
loss: 1131591.0 23
loss: 1130890.5 24
loss: 1130139.125 25
loss: 1129328.25 26
loss: 1128454.25 27
loss: 1127510.25 28
loss: 1126489.25 29
loss: 1125383.25 30
loss: 1124182.25 31
loss: 1122875.625 32
loss: 1121450.875 33
loss: 1119898.5 34
loss: 1118203.875 35
loss: 1116353.0 36
loss: 1114328.75 37
loss: 1112113.75 38
loss: 1109688.5 39
loss: 1107031.75 40
loss: 1104119.75 41
loss: 1100926.875 42
loss: 1097424.625 43
loss: 1093582.625 44
loss: 1089366.75 45
loss: 1084740.75 46
loss: 1079664.0 47
loss: 1074096.75 48
loss: 1067994.0 49
loss: 1061

loss: 12636.240234375 399
loss: 12634.501953125 400
loss: 12632.7724609375 401
loss: 12631.05859375 402
loss: 12629.35546875 403
loss: 12627.666015625 404
loss: 12625.9873046875 405
loss: 12624.318359375 406
loss: 12622.6728515625 407
loss: 12621.029296875 408
loss: 12619.3896484375 409
loss: 12617.76953125 410
loss: 12616.1640625 411
loss: 12614.560546875 412
loss: 12612.9697265625 413
loss: 12611.388671875 414
loss: 12609.814453125 415
loss: 12608.251953125 416
loss: 12606.6953125 417
loss: 12605.146484375 418
loss: 12603.6142578125 419
loss: 12602.08203125 420
loss: 12600.5625 421
loss: 12599.048828125 422
loss: 12597.5390625 423
loss: 12596.04296875 424
loss: 12594.548828125 425
loss: 12593.0625 426
loss: 12591.58203125 427
loss: 12590.1064453125 428
loss: 12588.6396484375 429
loss: 12587.1806640625 430
loss: 12585.7255859375 431
loss: 12584.279296875 432
loss: 12582.828125 433
loss: 12581.3916015625 434
loss: 12579.9619140625 435
loss: 12578.53515625 436
loss: 12577.1103515625 437

loss: 11514.1630859375 843
loss: 11508.900390625 844
loss: 11503.634765625 845
loss: 11498.314453125 846
loss: 11492.9794921875 847
loss: 11487.646484375 848
loss: 11482.2734375 849
loss: 11476.880859375 850
loss: 11471.443359375 851
loss: 11466.048828125 852
loss: 11460.5712890625 853
loss: 11455.0732421875 854
loss: 11449.630859375 855
loss: 11444.087890625 856
loss: 11438.56640625 857
loss: 11432.98046875 858
loss: 11427.435546875 859
loss: 11421.81640625 860
loss: 11416.18359375 861
loss: 11410.53515625 862
loss: 11404.8759765625 863
loss: 11399.1630859375 864
loss: 11393.478515625 865
loss: 11387.7314453125 866
loss: 11381.984375 867
loss: 11376.21875 868
loss: 11370.412109375 869
loss: 11364.60546875 870
loss: 11358.744140625 871
loss: 11352.87890625 872
loss: 11347.0009765625 873
loss: 11341.0791015625 874
loss: 11335.146484375 875
loss: 11329.20703125 876
loss: 11323.208984375 877
loss: 11317.228515625 878
loss: 11311.1728515625 879
loss: 11305.1318359375 880
loss: 11299.098632

loss: 6511.6103515625 1290
loss: 6495.20849609375 1291
loss: 6478.83740234375 1292
loss: 6462.53369140625 1293
loss: 6446.07958984375 1294
loss: 6429.650390625 1295
loss: 6413.3251953125 1296
loss: 6396.89111328125 1297
loss: 6380.3974609375 1298
loss: 6364.095703125 1299
loss: 6347.6435546875 1300
loss: 6331.095703125 1301
loss: 6314.82080078125 1302
loss: 6298.3203125 1303
loss: 6281.8115234375 1304
loss: 6265.47265625 1305
loss: 6248.96142578125 1306
loss: 6232.416015625 1307
loss: 6216.07275390625 1308
loss: 6199.5517578125 1309
loss: 6183.04443359375 1310
loss: 6166.6484375 1311
loss: 6150.12646484375 1312
loss: 6133.6279296875 1313
loss: 6117.1943359375 1314
loss: 6100.7236328125 1315
loss: 6084.1533203125 1316
loss: 6067.75732421875 1317
loss: 6051.1953125 1318
loss: 6034.68359375 1319
loss: 6018.251953125 1320
loss: 6001.716796875 1321
loss: 5985.189453125 1322
loss: 5968.818359375 1323
loss: 5952.2021484375 1324
loss: 5935.7041015625 1325
loss: 5919.3095703125 1326
loss: 5902.

loss: 1132.4296875 1734
loss: 1127.4112548828125 1735
loss: 1122.365966796875 1736
loss: 1117.313720703125 1737
loss: 1112.368408203125 1738
loss: 1107.4024658203125 1739
loss: 1102.43359375 1740
loss: 1097.5689697265625 1741
loss: 1092.6719970703125 1742
loss: 1087.7933349609375 1743
loss: 1083.015380859375 1744
loss: 1078.1702880859375 1745
loss: 1073.39599609375 1746
loss: 1068.671142578125 1747
loss: 1063.9215087890625 1748
loss: 1059.255615234375 1749
loss: 1054.615966796875 1750
loss: 1049.92724609375 1751
loss: 1045.34326171875 1752
loss: 1040.7762451171875 1753
loss: 1036.16748046875 1754
loss: 1031.650146484375 1755
loss: 1027.165771484375 1756
loss: 1022.6264038085938 1757
loss: 1018.19384765625 1758
loss: 1013.7694091796875 1759
loss: 1009.3319702148438 1760
loss: 1004.949462890625 1761
loss: 1000.6022338867188 1762
loss: 996.2527465820312 1763
loss: 991.92919921875 1764
loss: 987.6434326171875 1765
loss: 983.3846435546875 1766
loss: 979.1253051757812 1767
loss: 974.90393066

...

loss: 0.014095481485128403 41007
loss: 0.0140924621373415 41008
loss: 0.014088748954236507 41009
loss: 0.014084046706557274 41010
loss: 0.014081132598221302 41011
loss: 0.014077577739953995 41012
loss: 0.014074818231165409 41013
loss: 0.014071672223508358 41014
loss: 0.014067420735955238 41015
loss: 0.01406480185687542 41016
loss: 0.014059528708457947 41017
loss: 0.014056380838155746 41018
loss: 0.014051889069378376 41019
loss: 0.014050209894776344 41020
loss: 0.014047207310795784 41021
loss: 0.014043720439076424 41022
loss: 0.014040680602192879 41023
loss: 0.014037040993571281 41024
loss: 0.014034142717719078 41025
loss: 0.014030881226062775 41026
loss: 0.01402719784528017 41027
loss: 0.014023605734109879 41028
loss: 0.014020337723195553 41029
loss: 0.014016956090927124 41030
loss: 0.014014367014169693 41031
loss: 0.014010726474225521 41032
loss: 0.0140070216730237 41033
loss: 0.01400456391274929 41034
loss: 0.014001697301864624 41035
loss: 0.013998709619045258 41036
loss: 0.013995459

loss: 0.0131929200142622 41284
loss: 0.013188778422772884 41285
loss: 0.013185896910727024 41286
loss: 0.013181978836655617 41287
loss: 0.013179665431380272 41288
loss: 0.013175408355891705 41289
loss: 0.013173313811421394 41290
loss: 0.013169343583285809 41291
loss: 0.0131673663854599 41292
loss: 0.013164589181542397 41293
loss: 0.013162207789719105 41294
loss: 0.013157693669199944 41295
loss: 0.013154616579413414 41296
loss: 0.013151636347174644 41297
loss: 0.013148193247616291 41298
loss: 0.013144377619028091 41299
loss: 0.013141021132469177 41300
loss: 0.013138195499777794 41301
loss: 0.013134945183992386 41302
loss: 0.013132545165717602 41303
loss: 0.013130929321050644 41304
loss: 0.01312769204378128 41305
loss: 0.013125192373991013 41306
loss: 0.013122101314365864 41307
loss: 0.013119013980031013 41308
loss: 0.01311675738543272 41309
loss: 0.01311267726123333 41310
loss: 0.013110252097249031 41311
loss: 0.013107074424624443 41312
loss: 0.013104431331157684 41313
loss: 0.013100566

loss: 0.012193967588245869 41617
loss: 0.012191610410809517 41618
loss: 0.012188329361379147 41619
loss: 0.012185309082269669 41620
loss: 0.012182410806417465 41621
loss: 0.012178899720311165 41622
loss: 0.01217577327042818 41623
loss: 0.012173051945865154 41624
loss: 0.012171006761491299 41625
loss: 0.012167922221124172 41626
loss: 0.01216479204595089 41627
loss: 0.012162097729742527 41628
loss: 0.012159121222794056 41629
loss: 0.012156418524682522 41630
loss: 0.01215425692498684 41631
loss: 0.012150323018431664 41632
loss: 0.012147296220064163 41633
loss: 0.012143993750214577 41634
loss: 0.012140987440943718 41635
loss: 0.012138456106185913 41636
loss: 0.012135595083236694 41637
loss: 0.012132624164223671 41638
loss: 0.01213020272552967 41639
loss: 0.012127152644097805 41640
loss: 0.01212488953024149 41641
loss: 0.012121533043682575 41642
loss: 0.012119068764150143 41643
loss: 0.012116249650716782 41644
loss: 0.012113327160477638 41645
loss: 0.012111110612750053 41646
loss: 0.0121087

loss: 0.011001525446772575 42054
loss: 0.010999854654073715 42055
loss: 0.010996626690030098 42056
loss: 0.010995233431458473 42057
loss: 0.010991813614964485 42058
loss: 0.010988246649503708 42059
loss: 0.01098577305674553 42060
loss: 0.010983975604176521 42061
loss: 0.010980806313455105 42062
loss: 0.010978346690535545 42063
loss: 0.01097639836370945 42064
loss: 0.010973852127790451 42065
loss: 0.010970832780003548 42066
loss: 0.01096814963966608 42067
loss: 0.010965632274746895 42068
loss: 0.010962517000734806 42069
loss: 0.010959391482174397 42070
loss: 0.010956723242998123 42071
loss: 0.010954685509204865 42072
loss: 0.01095286849886179 42073
loss: 0.010949473828077316 42074
loss: 0.010946904309093952 42075
loss: 0.010944413021206856 42076
loss: 0.010942384600639343 42077
loss: 0.010939153842628002 42078
loss: 0.010937796905636787 42079
loss: 0.010935068130493164 42080
loss: 0.010932227596640587 42081
loss: 0.010929079726338387 42082
loss: 0.01092720776796341 42083
loss: 0.0109239

14
0.011782687964183311 0.174407210497557
15
0.020536685387005486 0.1607176348004666
16
0.2077814336945773 0.149934484845585
17
0.15462014787332254 0.15406640976337016
18
0.1764921907644012 0.15410332563736698
19
0.13132964219485652 0.15550262970780662
20
0.17896128878215556 0.15172459936749164
21
0.1729135669724166 0.15332675756835423
22
0.1754319272487665 0.15441491364635768


In [1]:
import sklearn